In [1]:
import pandas as pd

from sqlalchemy import create_engine
import mysql.connector

mysql_host = '34.116.84.145'
mysql_port = '3306'
mysql_user = 'gong-cha'
mysql_password = 'HelloGongCha2012'
mysql_database = 'gong_cha_redcat_db'

# Engine for MySQL
mysql_connection_string = f"mysql+mysqlconnector://{mysql_user}:{mysql_password}@{mysql_host}:{mysql_port}/{mysql_database}"
mysql_engine = create_engine(mysql_connection_string)

/var/folders/qw/dzbm6krn4k1gwl2301yj90f40000gn/T/ipykernel_27169/3170424087.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [5]:
# # # START OF FUNCTIONS

def extract_additional_hr(file, sheet_name):
  df = pd.read_excel(file, sheet_name = sheet_name)
  df = df.dropna(subset=['Employee ID'])

  if df['Employee ID'].dtypes == 'object':
    df['Employee ID'] = df['Employee ID'].str[:6]
  df['Employee ID'] = df['Employee ID'].astype(int)

  if df['Store'].dtypes == float:
    df['Store'] = df['Store'].astype(int).astype(str)

  df = df.fillna({'Add': 0, 'Add.1': 0,'Add.2': 0,'Add.3': 0,'Add.4': 0,'Add.5': 0,'Personal Leave': 0,'Annual Leave': 0,})
  df['2012-11-01'] = df['Add'] + df['Add.1'] + df['Add.2'] + df['Add.3'] + df['Add.4'] + df['Add.5']
  df['2013-01-01'] = df['Personal Leave'] + df['Annual Leave']
  cols =[0,1,2, df.columns.tolist().index('2012-11-01'),df.columns.tolist().index('2013-01-01')]

  df = df.iloc[:, cols]
  df = df.dropna(subset=df.columns[:3], how='all')
  df = df.melt(id_vars=['Employee ID', 'Store', 'Preferred Name'], value_vars=df.columns[3:], var_name='Date', value_name='Hours')
  df = df[df['Hours'] != 0]
  return df

def extract_rostered_hr(file, sheet_name):
  df = pd.read_excel(file, sheet_name = sheet_name)

  df = df.dropna(subset=['Employee ID'])

  if df['Employee ID'].dtypes == 'object':
    df['Employee ID'] = df['Employee ID'].str[:6]
  df['Employee ID'] = df['Employee ID'].astype(int)

  if df['Store'].dtypes == float:
    df['Store'] = df['Store'].astype(int).astype(str)

  df = df.iloc[:, :24]
  cols = [0,1,2]
  for col in range(3, 24, 3):
      df[df.columns[col]] = df[df.columns[col+2]]
      cols.append(col)
  df = df.iloc[:,cols]
  df = df.dropna(subset=df.columns[:3], how='all')
  df = df.melt(id_vars=['Employee ID', 'Store', 'Preferred Name'], value_vars=df.columns[3:], var_name='Date', value_name='Hours')
  df = df[df['Hours'] != 0]
  return df

def calc_timesheets_n_billings(files):
  print('calc')
  timesheets = pd.DataFrame()
  billings = pd.DataFrame()
  rostered_hr = pd.DataFrame()
  additional_hr = pd.DataFrame()

  for file in files:
    timesheet = pd.read_excel(file, sheet_name = 'Timesheet')
    billing = pd.read_excel(file, sheet_name = 'Billing')
    rostered_hr_w1 = extract_rostered_hr(file, 'Week 1 Roster')
    rostered_hr_w2 = extract_rostered_hr(file, 'Week 2 Roster')
    additional_hr_w1 = extract_additional_hr(file, 'Week 1 Roster')
    additional_hr_w2 = extract_additional_hr(file, 'Week 2 Roster')
    employees = pd.read_excel(file, sheet_name = 'Employees')

    timesheets = pd.concat([timesheets, timesheet], ignore_index=True)
    billings = pd.concat([billings, billing], ignore_index=True)
    rostered_hr = pd.concat([rostered_hr, rostered_hr_w1], ignore_index=True)
    rostered_hr = pd.concat([rostered_hr, rostered_hr_w2], ignore_index=True)
    additional_hr = pd.concat([additional_hr, additional_hr_w1], ignore_index=True)
    additional_hr = pd.concat([additional_hr, additional_hr_w2], ignore_index=True)

  #Remov  e irrelevant rows
  timesheets.dropna(subset = ['Employee ID'], inplace=True)
  #Keep the needed columns
  timesheets_cols = [1,2,3,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
  timesheets = timesheets[timesheets.columns[timesheets_cols]]
  timesheets['Update Wage'] = timesheets['Update Wage'].astype(bool)
  #Column Aggregations
  timesheets_agg_cols = {'First Name':'first','Last Name':'first','Update Wage':'first','Hour Threshold':'first','Company':'first','Ord':'sum','Sat':'sum','Sun':'sum','Pub':'sum','Eve 1':'sum','Eve 2':'sum','No. of Shifts':'sum','Personal Leave':'sum','Annual Leave':'sum','Unpaid Leave':'sum','Total':'sum'}
  timesheets = timesheets.groupby('Employee ID', as_index = False).agg(timesheets_agg_cols)
  #Calculate Over Threshold
  timesheets['Over Threshold'] = timesheets['Total'] - timesheets['Hour Threshold']
  timesheets.loc[timesheets["Over Threshold"] <=0, "Over Threshold"] = 0
  #Keep Over Thresholds to a new df
  over_threshold = timesheets[timesheets['Over Threshold']>0]
  #Reduce Ord & Total with the excess
  timesheets['Ord'] = timesheets['Ord'] - timesheets['Over Threshold']
  timesheets['Total'] = timesheets['Total'] - timesheets['Over Threshold']
  #Convert 80 & 100 hours to 76 hours
  hours_col = ['Ord', 'Sat','Sun','Eve 1','Eve 2','Pub','Personal Leave', 'Annual Leave', 'Unpaid Leave', 'Total']
  if(100 in timesheets["Hour Threshold"].values):
      timesheets.loc[timesheets["Hour Threshold"] == 100, hours_col] = timesheets[hours_col]/100*76
  if(80 in timesheets["Hour Threshold"].values):
      timesheets.loc[timesheets["Hour Threshold"] == 80, hours_col] = timesheets[hours_col]/80*76
  if any(timesheets["Hour Threshold"] > 1000):
  # Find rows where "Hour Threshold" is greater than 1000
      rows_to_update = timesheets.loc[timesheets["Hour Threshold"] > 1000]
      # Perform actions on the rows
      for index, row in rows_to_update.iterrows():
          threshold = int(row["Hour Threshold"])
          base = int(str(threshold)[:2])
          conversion = int(str(threshold)[-2:])
          # Update multiple columns using .loc
          timesheets.loc[index, hours_col] = timesheets.loc[index, hours_col] / conversion * base

  #drop Hour Threshold & Over Threshold
  timesheets = timesheets.drop(['Hour Threshold','Over Threshold'],axis = 1)

  #Remove irrelevant rows
  billings.dropna(subset=['Store'],inplace = True)
  billings = billings[billings['Total'] > 0]

  #Keep the needed columns
  billings_cols = [0,1,2,3,4,5,6,7,8,9,10,11]
  billings = billings[billings.columns[billings_cols]]
  #Column Aggregations
  billings_agg_cols = {'Ord':'sum','Sat':'sum','Sun':'sum','Pub':'sum','Eve 1':'sum','Eve 2':'sum','No. of Shifts':'sum','Personal Leave':'sum','Annual Leave':'sum','Unpaid Leave':'sum','Total':'sum'}
  billings = billings.groupby('Store', as_index = False).agg(billings_agg_cols)

  rostered_hr = pd.merge(rostered_hr, employees[['Employee ID', 'First Name', 'Last Name', 'Company']], how='left', on=['Employee ID'])
  rostered_hr_col = [
  'Employee ID',
  'First Name',
  'Last Name',
  'Preferred Name',
  'Company',
  'Store',
  'Date',
  'Hours'
  ]
  rostered_hr = rostered_hr[rostered_hr_col]
  rostered_hr['Date'] = pd.to_datetime(rostered_hr['Date'])

  bonus = rostered_hr.copy()

  # Stitch Store ID and drop rows which Store ID are not found
  sheet_id = '1rqOeBjA9drmTnjlENvr57RqL5-oxSqe_KGdbdL2MKhM'
  sheet_name = 'StoreReference'
  url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'
  store_ref = pd.read_csv(url)

  bonus = pd.merge(bonus, store_ref, on=['Store'], how = 'left')

  bonus.dropna(subset=['Store ID'], inplace = True)

  # Stitch recid_plo
  sheet_id = '1ezyBlKquUhYnFwmIKTR4fghI59ZvGaKL35mKbcdeRy4'
  sheet_name = 'Stores'
  url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'
  store_df_gs = pd.read_csv(url)

  bonus = pd.merge(bonus, store_df_gs[['Store ID', 'recid_plo']], on=['Store ID'], how = 'left')
  bonus['recid_plo'] = bonus['recid_plo'].astype(int)

  # Stich sales based on recid_plo & dates, skip if there is no Date
  start = bonus['Date'].min()
  end = bonus['Date'].max()

  start_str = start.strftime('%Y-%m-%d')
  end_str = end.strftime('%Y-%m-%d')

  recid_plo_list = bonus['recid_plo'].unique().tolist()
  recid_plo_list_str = ', '.join(str(id) for id in recid_plo_list)

  sheet_id = '1peA8effpeSTk3duIjxF46V-PrDD8tv3fubTCDEpD940'
  sheet_name = 'ops_bonus_exclusion'
  url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'
  exclusion_df = pd.read_csv(url)
  excluded_recid_plu = exclusion_df['recid_plu'].drop_duplicates()

  excluded_recid_plu_str = ', '.join(str(s) for s in excluded_recid_plu)
  excluded_recid_plu_str

  query = '''
  SELECT ts2.recid_plo, ts.itemdate as Date, sum(ts.qty*ts.price) as Sales
  FROM tbl_salesitems ts 
  JOIN tbl_salesheaders ts2 on ts.recid_mixh = ts2.recid
  WHERE ts.itemdate >= '{start}' and ts.itemdate <= '{end}' and ts2.recid_plo in ({recid_plo_list}) and ts.recid_plu not in ({excluded_recid_plu})
  GROUP BY ts2.recid_plo, ts.itemdate
  ORDER BY ts.itemdate ASC, recid_plo ASC
  '''.format(start=start_str, end = end_str, recid_plo_list = recid_plo_list_str, excluded_recid_plu = excluded_recid_plu_str)

  sales_df = pd.read_sql(query, mysql_engine)
  sales_df['Date'] = pd.to_datetime(sales_df['Date'])

  bonus = pd.merge(bonus, sales_df[['recid_plo', 'Date', 'Sales']], on=['recid_plo', 'Date'], how = 'left')

  # Stitch Target Sales & Bonus Rates
  sheet_id = '1rqOeBjA9drmTnjlENvr57RqL5-oxSqe_KGdbdL2MKhM'
  sheet_name = 'Targets'
  url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'
  targets = pd.read_csv(url)
  targets['Date'] = pd.to_datetime(targets['Date'])

  # Change Bonus Rates to 0, if Target Sales is not met
  bonus = pd.merge(bonus, targets[['Store ID', 'Date', 'Target Sales', 'Bonus Rate']], on=['Store ID', 'Date'], how = 'left')
  bonus['Bonus Rate'] = bonus['Bonus Rate'].where(bonus['Sales'] >= bonus['Target Sales'], 0)
  bonus['Bonus'] = bonus['Bonus Rate']  * bonus['Hours']

  # Work out additional_hr
  additional_hr = additional_hr.dropna(subset=['Employee ID'])
  additional_hr = pd.merge(additional_hr, employees[['Employee ID', 'First Name', 'Last Name', 'Company']], how='left', on=['Employee ID'])
  additional_hr_col = [
  'Employee ID',
  'First Name',
  'Last Name',
  'Preferred Name',
  'Company',
  'Store',
  'Date',
  'Hours'
  ]
  additional_hr = additional_hr[additional_hr_col]
  additional_hr['Date'] = pd.to_datetime(additional_hr['Date'])

  # Concat rostered_hr and additional_hr
  analysis = pd.concat([rostered_hr, additional_hr])

  # Concat Bouns on to Timesheets
  bonus_summary = bonus.groupby('Employee ID', as_index = False).agg({'Bonus':'sum'})
  timesheets = pd.merge(timesheets, bonus_summary, on=['Employee ID'], how = 'left')
  timesheets.fillna({'Bonus':0}, inplace = True)
  timesheets.rename(columns={'Bonus':'Bonus $'}, inplace = True)

  return timesheets, billings, over_threshold, analysis, bonus


# # # END OF FUNCTIONS


import io
import streamlit as st

st.title('Timesheet & Billing')

uploaded_files = st.file_uploader("Choose Files", accept_multiple_files = True)

# Create an empty container
output = st.empty()

# if uploaded_files is not None:
if len(uploaded_files) > 0:
    ts, bl, ot, an, bo = calc_timesheets_n_billings(uploaded_files)

    buffer = io.BytesIO()

    with pd.ExcelWriter(buffer, engine='xlsxwriter') as writer:
    # Write each dataframe to a different worksheet.
        ts.to_excel(writer, sheet_name='Timesheet', index = False)
        bl.to_excel(writer, sheet_name='Billing', index = False)
        ot.to_excel(writer, sheet_name='Over Threshold', index = False)
        an.to_excel(writer, sheet_name='Analysis',index = False)
        bo.to_excel(writer, sheet_name='Bonus',index = False)

    # Close the Pandas Excel writer and output the Excel file to the buffer
    writer.close()

    st.download_button(
        label="Download",
        data=buffer,
        file_name="Timesheet & Billing.xlsx",
        mime="application/vnd.ms-excel"
    )

ModuleNotFoundError: No module named 'streamlit'

In [8]:
tumbler_recid_plu_list = [1059,1060,1061,1062]
gingerbread_recid_plu_list = [485,559,572,573,574,575,1036,1085,1086]
stock_exclusions_list = tumbler_recid_plu_list + gingerbread_recid_plu_list
stock_exclusions_list_str = ', '.join(str(s) for s in stock_exclusions_list)



sheet_id = '1peA8effpeSTk3duIjxF46V-PrDD8tv3fubTCDEpD940'
sheet_name = 'ops_bonus_exclusion'
url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'
exclusion_df = pd.read_csv(url)
excluded_recid_plu = exclusion_df['recid_plu'].drop_duplicates()

excluded_recid_plu_str = ', '.join(str(s) for s in excluded_recid_plu)
excluded_recid_plu_str

'-2, 21, 568, 861, 570, 1024, 569, 1023, 845, 846, 847, 848, 1059, 1060, 1061, 1062, 485, 559, 572, 573, 574, 575, 892, 1036, 1085, 1086, 1133, 1135, 1136'

In [6]:
stock_exclusions_list_str

'1059, 1060, 1061, 1062, 485, 559, 572, 573, 574, 575, 1036, 1085, 1086'

In [29]:
import os
import openpyxl

# Get absolute path of the current notebook's directory
notebook_dir = os.getcwd()  # Get current working directory

# Construct the path to the `/files/` folder
files_dir = os.path.join(notebook_dir, "files")

# List files in the directory
files_names_dir = os.listdir(files_dir)

timesheets = pd.DataFrame()
billings = pd.DataFrame()
rostered_hr = pd.DataFrame()
additional_hr = pd.DataFrame()

for filename in files_names_dir:
    if filename.endswith(".xlsx"):
        file = os.path.join(files_dir, filename)
        print(file)
        timesheet = pd.read_excel(file, sheet_name = 'Timesheet', engine='openpyxl')
        billing = pd.read_excel(file, sheet_name = 'Billing', engine='openpyxl')
        rostered_hr_w1 = extract_rostered_hr(file, 'Week 1 Roster')
        rostered_hr_w2 = extract_rostered_hr(file, 'Week 2 Roster')
        additional_hr_w1 = extract_additional_hr(file, 'Week 1 Roster')
        additional_hr_w2 = extract_additional_hr(file, 'Week 2 Roster')
        employees = pd.read_excel(file, sheet_name = 'Employees')

        timesheets = pd.concat([timesheets, timesheet], ignore_index=True)
        billings = pd.concat([billings, billing], ignore_index=True)
        rostered_hr = pd.concat([rostered_hr, rostered_hr_w1], ignore_index=True)
        rostered_hr = pd.concat([rostered_hr, rostered_hr_w2], ignore_index=True)
        additional_hr = pd.concat([additional_hr, additional_hr_w1], ignore_index=True)
        additional_hr = pd.concat([additional_hr, additional_hr_w2], ignore_index=True)

#Remov  e irrelevant rows
timesheets.dropna(subset = ['Employee ID'], inplace=True)
#Keep the needed columns
timesheets_cols = [1,2,3,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
timesheets = timesheets[timesheets.columns[timesheets_cols]]
timesheets['Update Wage'] = timesheets['Update Wage'].astype(bool)
#Column Aggregations
timesheets_agg_cols = {'First Name':'first','Last Name':'first','Update Wage':'first','Hour Threshold':'first','Company':'first','Ord':'sum','Sat':'sum','Sun':'sum','Pub':'sum','Eve 1':'sum','Eve 2':'sum','No. of Shifts':'sum','Personal Leave':'sum','Annual Leave':'sum','Unpaid Leave':'sum','Total':'sum'}
timesheets = timesheets.groupby('Employee ID', as_index = False).agg(timesheets_agg_cols)
#Calculate Over Threshold
timesheets['Over Threshold'] = timesheets['Total'] - timesheets['Hour Threshold']
timesheets.loc[timesheets["Over Threshold"] <=0, "Over Threshold"] = 0
#Keep Over Thresholds to a new df
over_threshold = timesheets[timesheets['Over Threshold']>0]
#Reduce Ord & Total with the excess
timesheets['Ord'] = timesheets['Ord'] - timesheets['Over Threshold']
timesheets['Total'] = timesheets['Total'] - timesheets['Over Threshold']
#Convert 80 & 100 hours to 76 hours
hours_col = ['Ord', 'Sat','Sun','Eve 1','Eve 2','Pub','Personal Leave', 'Annual Leave', 'Unpaid Leave', 'Total']
if(100 in timesheets["Hour Threshold"].values):
    timesheets.loc[timesheets["Hour Threshold"] == 100, hours_col] = timesheets[hours_col]/100*76
if(80 in timesheets["Hour Threshold"].values):
    timesheets.loc[timesheets["Hour Threshold"] == 80, hours_col] = timesheets[hours_col]/80*76
if any(timesheets["Hour Threshold"] > 1000):
# Find rows where "Hour Threshold" is greater than 1000
    rows_to_update = timesheets.loc[timesheets["Hour Threshold"] > 1000]
    # Perform actions on the rows
    for index, row in rows_to_update.iterrows():
        threshold = int(row["Hour Threshold"])
        base = int(str(threshold)[:2])
        conversion = int(str(threshold)[-2:])
        # Update multiple columns using .loc
        timesheets.loc[index, hours_col] = timesheets.loc[index, hours_col] / conversion * base

#drop Hour Threshold & Over Threshold
timesheets = timesheets.drop(['Hour Threshold','Over Threshold'],axis = 1)

#Remove irrelevant rows
billings.dropna(subset=['Store'],inplace = True)
billings = billings[billings['Total'] > 0]

#Keep the needed columns
billings_cols = [0,1,2,3,4,5,6,7,8,9,10,11]
billings = billings[billings.columns[billings_cols]]
#Column Aggregations
billings_agg_cols = {'Ord':'sum','Sat':'sum','Sun':'sum','Pub':'sum','Eve 1':'sum','Eve 2':'sum','No. of Shifts':'sum','Personal Leave':'sum','Annual Leave':'sum','Unpaid Leave':'sum','Total':'sum'}
billings = billings.groupby('Store', as_index = False).agg(billings_agg_cols)

rostered_hr = pd.merge(rostered_hr, employees[['Employee ID', 'First Name', 'Last Name', 'Company']], how='left', on=['Employee ID'])
rostered_hr_col = [
'Employee ID',
'First Name',
'Last Name',
'Preferred Name',
'Company',
'Store',
'Date',
'Hours'
]
rostered_hr = rostered_hr[rostered_hr_col]
rostered_hr['Date'] = pd.to_datetime(rostered_hr['Date'])

bonus = rostered_hr.copy()

# Stitch Store ID and drop rows which Store ID are not found
sheet_id = '1rqOeBjA9drmTnjlENvr57RqL5-oxSqe_KGdbdL2MKhM'
sheet_name = 'StoreReference'
url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'
store_ref = pd.read_csv(url)

bonus = pd.merge(bonus, store_ref, on=['Store'], how = 'left')

bonus.dropna(subset=['Store ID'], inplace = True)

# Stitch recid_plo
sheet_id = '1ezyBlKquUhYnFwmIKTR4fghI59ZvGaKL35mKbcdeRy4'
sheet_name = 'Stores'
url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'
store_df_gs = pd.read_csv(url)

bonus = pd.merge(bonus, store_df_gs[['Store ID', 'recid_plo']], on=['Store ID'], how = 'left')
bonus['recid_plo'] = bonus['recid_plo'].astype(int)

# Stich sales based on recid_plo & dates, skip if there is no Date
start = bonus['Date'].min()
end = bonus['Date'].max()

start_str = start.strftime('%Y-%m-%d')
end_str = end.strftime('%Y-%m-%d')

recid_plo_list = bonus['recid_plo'].unique().tolist()
recid_plo_list_str = ', '.join(str(id) for id in recid_plo_list)

tumbler_recid_plu_list = [1059,1060,1061,1062]
gingerbread_recid_plu_list = [485,559,572,573,574,575,1036,1085,1086]
stock_exclusions_list = tumbler_recid_plu_list + gingerbread_recid_plu_list
stock_exclusions_list_str = ', '.join(str(s) for s in stock_exclusions_list)

query = '''
SELECT ts2.recid_plo, ts.itemdate as Date, sum(ts.qty*ts.price) as Sales
FROM tbl_salesitems ts 
JOIN tbl_salesheaders ts2 on ts.recid_mixh = ts2.recid
WHERE ts.itemdate >= '{start}' and ts.itemdate <= '{end}' and ts2.recid_plo in ({recid_plo_list}) and ts.recid_plu not in ({stock_exclusions_list})
GROUP BY ts2.recid_plo, ts.itemdate
ORDER BY ts.itemdate ASC, recid_plo ASC
'''.format(start=start_str, end = end_str, recid_plo_list = recid_plo_list_str, stock_exclusions_list = stock_exclusions_list_str)

sales_df = pd.read_sql(query, mysql_engine)
sales_df['Date'] = pd.to_datetime(sales_df['Date'])

bonus = pd.merge(bonus, sales_df[['recid_plo', 'Date', 'Sales']], on=['recid_plo', 'Date'], how = 'left')

# Stitch Target Sales & Bonus Rates
sheet_id = '1rqOeBjA9drmTnjlENvr57RqL5-oxSqe_KGdbdL2MKhM'
sheet_name = 'Targets'
url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'
targets = pd.read_csv(url)
targets['Date'] = pd.to_datetime(targets['Date'])

# Change Bonus Rates to 0, if Target Sales is not met
bonus = pd.merge(bonus, targets[['Store ID', 'Date', 'Target Sales', 'Bonus Rate']], on=['Store ID', 'Date'], how = 'left')
bonus['Bonus Rate'] = bonus['Bonus Rate'].where(bonus['Sales'] >= bonus['Target Sales'], 0)
bonus['Bonus'] = bonus['Bonus Rate']  * bonus['Hours']

# Work out additional_hr
additional_hr = additional_hr.dropna(subset=['Employee ID'])
additional_hr = pd.merge(additional_hr, employees[['Employee ID', 'First Name', 'Last Name', 'Company']], how='left', on=['Employee ID'])
additional_hr_col = [
'Employee ID',
'First Name',
'Last Name',
'Preferred Name',
'Company',
'Store',
'Date',
'Hours'
]
additional_hr = additional_hr[additional_hr_col]
additional_hr['Date'] = pd.to_datetime(additional_hr['Date'])

# Concat rostered_hr and additional_hr
analysis = pd.concat([rostered_hr, additional_hr])

# Concat Bouns on to Timesheets
bonus_summary = bonus.groupby('Employee ID', as_index = False).agg({'Bonus':'sum'})
timesheets = pd.merge(timesheets, bonus_summary, on=['Employee ID'], how = 'left')
timesheets.fillna({'Bonus':0}, inplace = True)
timesheets.rename(columns={'Bonus':'Bonus $'}, inplace = True)





/Users/eddy/Developer/Streamlit/gongchaaus/roster-consolidation/files/501 Final Roster 0502-1802.xlsx


/opt/homebrew/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
/opt/homebrew/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
/opt/homebrew/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
/opt/homebrew/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [30]:
timesheets

,Employee ID,First Name,Last Name,Update Wage,Company,Ord,Sat,Sun,Pub,Eve 1,Eve 2,No. of Shifts,Personal Leave,Annual Leave,Unpaid Leave,Total,Bonus $
0,10008.0,Elsie,Chan,False,GCM,15.250000,19.000000,0.066667,0,8.00,0.050000,6,0,0.00,0,42.366667,0.0
1,10010.0,Erica Fong-de,Jong,False,GCM,29.000000,6.500000,19.500000,0,0.00,0.000000,6,0,0.00,0,55.000000,0.0
2,10013.0,Hanbo,Zhang,False,GCM,3.500000,0.000000,0.000000,0,1.50,0.000000,1,0,0.00,0,5.000000,0.0
3,10032.0,Reynardo,Tjhin,False,GCM,28.500000,17.083333,0.183333,0,7.00,0.083333,6,0,0.00,0,52.850000,0.0
4,10054.0,Yiran,Zhang,False,SS,4.000000,0.000000,0.000000,0,1.00,0.000000,1,0,0.00,0,5.000000,0.0
5,10058.0,Han-Chun,Lin,False,GCM,34.130000,6.500000,0.000000,0,4.00,0.000000,4,0,31.37,0,76.000000,0.0
6,10061.0,Adrian,Lim,False,HL,17.000000,0.000000,0.000000,0,1.50,0.000000,2,0,0.00,0,18.500000,0.0
7,10081.0,Emmanuel,Quebec,False,HL,0.000000,0.000000,8.500000,0,0.00,0.000000,1,0,0.00,0,8.500000,0.0
8,10114.0,Mari,Cuasay,False,HL,0.000000,0.000000,11.000000,0,0.00,0.000000,1,0,0.00,0,11.000000,0.0
9,10120.0,Namrata,Gurung,False,HL,70.110000,0.000000,0.000000,0,4.56,0.215333,9,0,0.00,0,74.885333,0.0


In [62]:
bonus

,Employee ID,First Name,Last Name,Preferred Name,Company,Store,Date,Hours,Store ID,recid_plo,Sales,Target Sales,Bonus Rate,Bonus
0,10035,Shanzhi,Chen,CHLOE CHEN,GCM,Waterloo,2024-01-22,8.5,S164,154,NaN,724.06,0.0,0.0
1,10035,Shanzhi,Chen,CHLOE CHEN,GCM,Waterloo,2024-01-22,8.5,S164,154,NaN,784.39,0.0,0.0
2,10052,Yeyu,Deng,DEREK DENG,GCM,Waterloo,2024-01-22,6.0,S164,154,NaN,724.06,0.0,0.0
3,10052,Yeyu,Deng,DEREK DENG,GCM,Waterloo,2024-01-22,6.0,S164,154,NaN,784.39,0.0,0.0
4,10299,YUTANG,HUANG,RYAN HUANG,GCM,Waterloo,2024-01-23,7.5,S164,154,NaN,978.01,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1803,10343,Theresa,Chow,Theresa chow,SS,501,2024-02-04,6.5,S001,124,5592.42,8213.80,0.0,0.0
1804,10054,Yiran,Zhang,Eos zhang,SS,501,2024-02-04,7.5,S001,124,5592.42,7581.97,0.0,0.0
1805,10054,Yiran,Zhang,Eos zhang,SS,501,2024-02-04,7.5,S001,124,5592.42,8213.80,0.0,0.0
1806,10334,Huy Tien,Phan,Pat phan,SS,501,2024-02-04,7.5,S001,124,5592.42,7581.97,0.0,0.0


,Employee ID,First Name,Last Name,Preferred Name,Company,Store,Date,Hours,Store ID,recid_plo
0,10035,Shanzhi,Chen,CHLOE CHEN,GCM,Waterloo,2024-01-22,8.5,S164,154
1,10052,Yeyu,Deng,DEREK DENG,GCM,Waterloo,2024-01-22,6.0,S164,154
2,10299,YUTANG,HUANG,RYAN HUANG,GCM,Waterloo,2024-01-23,7.5,S164,154
3,10052,Yeyu,Deng,DEREK DENG,GCM,Waterloo,2024-01-23,7.0,S164,154
4,10052,Yeyu,Deng,DEREK DENG,GCM,Waterloo,2024-01-24,14.5,S164,154
...,...,...,...,...,...,...,...,...,...,...
917,10276,Cana,Fukuda,Cana fukuda,GCM,501,2024-02-04,9.0,S001,124
918,10303,Trong Van,Le,Van le,HL,501,2024-02-04,5.0,S001,124
919,10343,Theresa,Chow,Theresa chow,SS,501,2024-02-04,6.5,S001,124
920,10054,Yiran,Zhang,Eos zhang,SS,501,2024-02-04,7.5,S001,124


In [47]:
bonus

,Employee ID,First Name,Last Name,Preferred Name,Company,Store,Date,Hours,Store ID,recid_plo,Sales
0,10035,Shanzhi,Chen,CHLOE CHEN,GCM,Waterloo,2024-01-22,8.5,S164,154,NaN
1,10052,Yeyu,Deng,DEREK DENG,GCM,Waterloo,2024-01-22,6.0,S164,154,NaN
2,10299,YUTANG,HUANG,RYAN HUANG,GCM,Waterloo,2024-01-23,7.5,S164,154,NaN
3,10052,Yeyu,Deng,DEREK DENG,GCM,Waterloo,2024-01-23,7.0,S164,154,NaN
4,10052,Yeyu,Deng,DEREK DENG,GCM,Waterloo,2024-01-24,14.5,S164,154,NaN
...,...,...,...,...,...,...,...,...,...,...,...
917,10276,Cana,Fukuda,Cana fukuda,GCM,501,2024-02-04,9.0,S001,124,5592.42
918,10303,Trong Van,Le,Van le,HL,501,2024-02-04,5.0,S001,124,5592.42
919,10343,Theresa,Chow,Theresa chow,SS,501,2024-02-04,6.5,S001,124,5592.42
920,10054,Yiran,Zhang,Eos zhang,SS,501,2024-02-04,7.5,S001,124,5592.42


In [ ]:

def calc_timesheets_n_billings(files):
  print('calc')
  timesheets = pd.DataFrame()
  billings = pd.DataFrame()
  rostered_hr = pd.DataFrame()
  additional_hr = pd.DataFrame()

  for file in files:
    # time.sleep(1)
    timesheet = pd.read_excel(file, sheet_name = 'Timesheet')
    billing = pd.read_excel(file, sheet_name = 'Billing')
    rostered_hr_w1 = extract_rostered_hr(file, 'Week 1 Roster')
    rostered_hr_w2 = extract_rostered_hr(file, 'Week 2 Roster')
    additional_hr_w1 = extract_additional_hr(file, 'Week 1 Roster')
    additional_hr_w2 = extract_additional_hr(file, 'Week 2 Roster')
    employees = pd.read_excel(file, sheet_name = 'Employees')

    timesheets = pd.concat([timesheets, timesheet], ignore_index=True)
    billings = pd.concat([billings, billing], ignore_index=True)
    rostered_hr = pd.concat([rostered_hr, rostered_hr_w1], ignore_index=True)
    rostered_hr = pd.concat([rostered_hr, rostered_hr_w2], ignore_index=True)
    additional_hr = pd.concat([additional_hr, additional_hr_w1], ignore_index=True)
    additional_hr = pd.concat([additional_hr, additional_hr_w2], ignore_index=True)

  #Remove irrelevant rows
  timesheets.dropna(subset = ['Employee ID'], inplace=True)
  #Keep the needed columns
  timesheets_cols = [1,2,3,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
  timesheets = timesheets[timesheets.columns[timesheets_cols]]
  timesheets['Update Wage'] = timesheets['Update Wage'].astype(bool)
  #Column Aggregations
  timesheets_agg_cols = {'First Name':'first','Last Name':'first','Update Wage':'first','Hour Threshold':'first','Company':'first','Ord':'sum','Sat':'sum','Sun':'sum','Pub':'sum','Eve 1':'sum','Eve 2':'sum','No. of Shifts':'sum','Personal Leave':'sum','Annual Leave':'sum','Unpaid Leave':'sum','Total':'sum'}
  timesheets = timesheets.groupby('Employee ID', as_index = False).agg(timesheets_agg_cols)
  #Calculate Over Threshold
  timesheets['Over Threshold'] = timesheets['Total'] - timesheets['Hour Threshold']
  timesheets.loc[timesheets["Over Threshold"] <=0, "Over Threshold"] = 0
  #Keep Over Thresholds to a new df
  over_threshold = timesheets[timesheets['Over Threshold']>0]
  #Reduce Ord & Total with the excess
  timesheets['Ord'] = timesheets['Ord'] - timesheets['Over Threshold']
  timesheets['Total'] = timesheets['Total'] - timesheets['Over Threshold']
  #Convert 80 & 100 hours to 76 hours
  hours_col = ['Ord', 'Sat','Sun','Eve 1','Eve 2','Pub','Personal Leave', 'Annual Leave', 'Unpaid Leave', 'Total']
  if(100 in timesheets["Hour Threshold"].values):
    timesheets.loc[timesheets["Hour Threshold"] == 100, hours_col] = timesheets[hours_col]/100*76
  if(80 in timesheets["Hour Threshold"].values):
    timesheets.loc[timesheets["Hour Threshold"] == 80, hours_col] = timesheets[hours_col]/80*76
  if any(timesheets["Hour Threshold"] > 1000):
    # Find rows where "Hour Threshold" is greater than 1000
    rows_to_update = timesheets.loc[timesheets["Hour Threshold"] > 1000]
    # Perform actions on the rows
    for index, row in rows_to_update.iterrows():
      threshold = int(row["Hour Threshold"])
      base = int(str(threshold)[:2])
      # print('before conversion')
      conversion = int(str(threshold)[-2:])
      # print('after conversion')
      # Update multiple columns using .loc
      timesheets.loc[index, hours_col] = timesheets.loc[index, hours_col] / conversion * base

  #drop Hour Threshold & Over Threshold
  timesheets = timesheets.drop(['Hour Threshold','Over Threshold'],axis = 1)

  #Remove irrelevant rows
  billings.dropna(subset=['Store'],inplace = True)
  billings = billings[billings['Total'] > 0]

  #Keep the needed columns
  billings_cols = [0,1,2,3,4,5,6,7,8,9,10,11]
  billings = billings[billings.columns[billings_cols]]
  #Column Aggregations
  billings_agg_cols = {'Ord':'sum','Sat':'sum','Sun':'sum','Pub':'sum','Eve 1':'sum','Eve 2':'sum','No. of Shifts':'sum','Personal Leave':'sum','Annual Leave':'sum','Unpaid Leave':'sum','Total':'sum'}
  billings = billings.groupby('Store', as_index = False).agg(billings_agg_cols)

  rostered_hr = pd.merge(rostered_hr, employees[['Employee ID', 'First Name', 'Last Name', 'Company']], how='left', on=['Employee ID'])
  rostered_hr_col = [
    'Employee ID',
    'First Name',
    'Last Name',
    'Preferred Name',
    'Company',
    'Store',
    'Date',
    'Hours'
    ]
  rostered_hr = rostered_hr[rostered_hr_col]
  rostered_hr['Date'] = pd.to_datetime(rostered_hr['Date'])

  bonus = rostered_hr.copy()

  # Stitch Store ID and drop rows which Store ID are not found
  sheet_id = '1rqOeBjA9drmTnjlENvr57RqL5-oxSqe_KGdbdL2MKhM'
  sheet_name = 'StoreReference'
  url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'
  store_ref = pd.read_csv(url)

  bonus = pd.merge(bonus, store_ref, on=['Store'], how = 'left')

  bonus.dropna(subset=['Store ID'], inplace = True)

  # Stitch shop_id
  sheet_id = '1ezyBlKquUhYnFwmIKTR4fghI59ZvGaKL35mKbcdeRy4'
  sheet_name = 'Stores'
  url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'
  store_df_gs = pd.read_csv(url)

  bonus = pd.merge(bonus, store_df_gs[['Store ID', 'shop_id']], on=['Store ID'], how = 'left')
  bonus['shop_id'] = bonus['shop_id'].astype(int)


  # Stich sales based on shop_id & dates, skip if there is no Date
  start = bonus['Date'].min()
  end = bonus['Date'].max()

  start_str = start.strftime('%Y-%m-%d')
  end_str = (end+timedelta(days=1)).strftime('%Y-%m-%d')
  shop_id_list = bonus['shop_id'].unique().tolist()
  shop_id_list_str = ', '.join(str(id) for id in shop_id_list)
  stock_exclusions_list = [266,267,268,269,270,272,256,255,254]
  stock_exclusions_list_str = ', '.join(str(s) for s in stock_exclusions_list)

  query = '''
  SELECT
    d.shop_id, Date(d.docket_date) AS Date, SUM(dl.quantity * dl.sell_inc) as Sales
  FROM
    DocketLine dl
  JOIN
    Docket d on dl.docket_id  = d.docket_id 
  WHERE 
    d.docket_date >='{start}' and d.docket_date < '{end}' and d.shop_id in ({shop_id_list}) and d.`transaction`  = 'SA' and dl.stock_id not in ({stock_exclusions_list})
  GROUP BY 
    d.shop_id, Date(d.docket_date)
  '''.format(start=start_str, end = end_str, shop_id_list = shop_id_list_str, stock_exclusions_list = stock_exclusions_list_str)
  sales_df = pd.read_sql(con=Gong_cha_MySQL_engine.connect(), sql=sql_text(query))
  sales_df['Date'] = pd.to_datetime(sales_df['Date'])

  bonus = pd.merge(bonus, sales_df[['shop_id', 'Date', 'Sales']], on=['shop_id', 'Date'], how = 'left')

  # Stitch Target Sales & Bonus Rates
  sheet_id = '1rqOeBjA9drmTnjlENvr57RqL5-oxSqe_KGdbdL2MKhM'
  sheet_name = 'Targets'
  url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'
  targets = pd.read_csv(url)
  targets['Date'] = pd.to_datetime(targets['Date'])

  # Change Bonus Rates to 0, if Target Sales is not met
  bonus = pd.merge(bonus, targets[['Store ID', 'Date', 'Target Sales', 'Bonus Rate']], on=['Store ID', 'Date'], how = 'left')
  bonus['Bonus Rate'] = bonus['Bonus Rate'].where(bonus['Sales'] >= bonus['Target Sales'], 0)
  bonus['Bonus'] = bonus['Bonus Rate']  * bonus['Hours']

  # Work out additional_hr
  additional_hr = additional_hr.dropna(subset=['Employee ID'])
  additional_hr = pd.merge(additional_hr, employees[['Employee ID', 'First Name', 'Last Name', 'Company']], how='left', on=['Employee ID'])
  additional_hr_col = [
    'Employee ID',
    'First Name',
    'Last Name',
    'Preferred Name',
    'Company',
    'Store',
    'Date',
    'Hours'
  ]
  additional_hr = additional_hr[additional_hr_col]
  additional_hr['Date'] = pd.to_datetime(additional_hr['Date'])

  # Concat rostered_hr and additional_hr
  analysis = pd.concat([rostered_hr, additional_hr])

  # Concat Bouns on to Timesheets
  bonus_summary = bonus.groupby('Employee ID', as_index = False).agg({'Bonus':'sum'})
  timesheets = pd.merge(timesheets, bonus_summary, on=['Employee ID'], how = 'left')
  timesheets.fillna({'Bonus':0}, inplace = True)
  timesheets.rename(columns={'Bonus':'Bonus $'}, inplace = True)

  return timesheets, billings, over_threshold, analysis, bonus

# # # END OF FUNCTIONS


import io
import streamlit as st

st.title('Timesheet & Billing')

uploaded_files = st.file_uploader("Choose Files", accept_multiple_files = True)

# Create an empty container
output = st.empty()

# if uploaded_files is not None:
if len(uploaded_files) > 0:
    ts, bl, ot, an, bo = calc_timesheets_n_billings(uploaded_files)

    buffer = io.BytesIO()

    with pd.ExcelWriter(buffer, engine='xlsxwriter') as writer:
    # Write each dataframe to a different worksheet.
        ts.to_excel(writer, sheet_name='Timesheet', index = False)
        bl.to_excel(writer, sheet_name='Billing', index = False)
        ot.to_excel(writer, sheet_name='Over Threshold', index = False)
        an.to_excel(writer, sheet_name='Analysis',index = False)
        bo.to_excel(writer, sheet_name='Bonus',index = False)

    # Close the Pandas Excel writer and output the Excel file to the buffer
    writer.close()

    st.download_button(
        label="Download",
        data=buffer,
        file_name="Timesheet & Billing.xlsx",
        mime="application/vnd.ms-excel"
    )
